# @chain 데코레이터로 명령형 체인 구성하기

이 노트북에서는 **@chain 데코레이터**를 사용하여 명령형(Imperative) 방식으로 체인을 구성하는 방법을 알아봅니다.

## 선언형 vs 명령형 체인

LangChain에서 체인을 구성하는 두 가지 방식이 있습니다:

| 방식 | 문법 | 특징 |
|------|------|------|
| **선언형 (LCEL)** | `chain = a \| b \| c` | 파이프 연산자, 간결함 |
| **명령형 (Imperative)** | `@chain` 데코레이터 + 함수 | 세밀한 제어, 조건문/반복문 사용 가능 |

## @chain 데코레이터란?

일반 Python 함수에 **Runnable 인터페이스**를 추가하는 데코레이터입니다.

```python
from langchain_core.runnables import chain

@chain
def my_chain(inputs):
    # 자유로운 Python 코드
    return result

# Runnable 메서드 사용 가능!
my_chain.invoke(inputs)
my_chain.batch([inputs1, inputs2])
my_chain.stream(inputs)
```

## 언제 명령형을 사용할까?

1. **조건부 로직**: 입력에 따라 다른 처리가 필요할 때
2. **복잡한 흐름**: 반복문, 예외 처리 등이 필요할 때
3. **중간 처리**: 단계 사이에 데이터 가공이 필요할 때
4. **디버깅**: 각 단계를 명시적으로 제어하고 싶을 때

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 2s (343 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current use

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. 구성 요소 준비

체인에서 사용할 **Prompt Template**과 **Model**을 정의합니다.

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# Prompt Template
template = ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 어시스턴트입니다.'),
    ('human', '{question}'),
])

# Model
model = ChatOllama(model='llama3.2')

# 4. @chain 데코레이터로 체인 구성

**코드 설명:**

### @chain 데코레이터 적용
```python
@chain
def chatbot(values):
    prompt = template.invoke(values)  # Step 1: 프롬프트 생성
    return model.invoke(prompt)        # Step 2: 모델 호출
```

**핵심 포인트:**
- `@chain` 데코레이터로 일반 함수를 **Runnable**로 변환
- 함수 내부에서 **명시적으로** 각 단계 실행
- `invoke()`, `batch()`, `stream()` 메서드 자동 지원

### LCEL 방식과 비교
```python
# LCEL (선언형)
chain = template | model

# @chain (명령형)
@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)
```

결과는 동일하지만, 명령형은 **중간에 자유로운 코드 삽입**이 가능합니다.

In [4]:
from langchain_core.runnables import chain

# @chain 데코레이터로 Runnable 함수 생성
@chain
def chatbot(values):
    prompt = template.invoke(values)  # Step 1: 프롬프트 생성
    return model.invoke(prompt)        # Step 2: 모델 호출

# 5. 체인 실행

`@chain`으로 만든 함수는 **Runnable 인터페이스**를 갖습니다.

- `chatbot.invoke()` - 단일 실행
- `chatbot.batch()` - 배치 실행
- `chatbot.stream()` - 스트리밍

In [5]:
# invoke() 실행
response = chatbot.invoke({'question': '거대 언어 모델은 어디서 제공하나요?'})

print("=== chatbot.invoke() 결과 ===")
print(response.content)

=== chatbot.invoke() 결과 ===
거대 언어 모델은 다양한แหล에서 사용할 수 있습니다. 가장 잘-known한 거대 언어 모델은 다음의 곳에서 다운로드 할 수 있습니다.

1.  **Hugging Face** : Hugging Face는 거대 언어 모델을 포함한 많은 자연어 처리 라이브러리와 모델을 제공합니다.
2.  **GitHub** : GitHub는 다양한 거대 언어 모델을 open-source로 제공하는곳입니다.
3.  **Google** : Google은 TensorFlow와 같은 자연어 처리 libraries를 제공하며, 거대 언어 모델을 포함하여 다양한 models를 제공합니다.
4.  **Amazon SageMaker** : Amazon SageMaker는 인공지능을 위한 개발 및 배포 platform이며, 거대 언어 모델을 포함하여 다양한 models를 제공합니다.

이러한 곳에서 제공하는 거대 언어 모델은 여러가지의 종류가 있으며, 각종의 사용에เหมาะ합니다.


In [6]:
# batch() 실행
responses = chatbot.batch([
    {'question': 'Python이 뭔가요?'},
    {'question': 'LangChain이 뭔가요?'}
])

print("=== chatbot.batch() 결과 ===")
for i, resp in enumerate(responses):
    print(f"\n[{i+1}] {resp.content[:100]}...")

=== chatbot.batch() 결과 ===

[1] Python은 인공 지능, 데이터 과학,-web 개발 및 otros 여러 분야에서 널리 사용되는 프로그래밍 언어입니다.

Python은 1991년에 Guido van Rossum가...

[2] LangChain은 Python로 작성된 도구이지만, 여러 프로그래밍 언어를 지원하는-chain-of-actions API입니다. 
 LangChain은 ChainOfActions...


# 6. 명령형의 장점: 조건부 로직

명령형 방식의 가장 큰 장점은 **조건문, 반복문** 등을 자유롭게 사용할 수 있다는 것입니다.

In [7]:
# 조건부 로직이 포함된 체인
@chain
def smart_chatbot(values):
    question = values.get('question', '')

    # 조건에 따라 다른 시스템 메시지 사용
    if '코드' in question or '프로그래밍' in question:
        system_msg = '당신은 프로그래밍 전문가입니다. 코드 예시와 함께 설명하세요.'
    elif '번역' in question:
        system_msg = '당신은 전문 번역가입니다.'
    else:
        system_msg = '당신은 친절한 어시스턴트입니다.'

    # 동적으로 템플릿 생성
    dynamic_template = ChatPromptTemplate.from_messages([
        ('system', system_msg),
        ('human', '{question}'),
    ])

    prompt = dynamic_template.invoke(values)
    return model.invoke(prompt)

# 테스트
print("=== 조건부 체인 테스트 ===")
response = smart_chatbot.invoke({'question': 'Python 코드로 Hello World 출력하는 방법'})
print(response.content)

=== 조건부 체인 테스트 ===
Python에서 "Hello, World!"를 출력하는 simplest way은 `print()` 함수를 사용합니다.

```python
# hello_world.py

def main():
    # print() 함수는 그 안에 문자열을 입력해주면 해당 문자열이 화면에 표시됩니다.
    print("Hello, World!")

if __name__ == "__main__":
    main()
```

이 코드는 `print()` 함수를 사용하여 "Hello, World!"를화면에 출력하는 것입니다. 

**Python의 가상 환경을 사용하고 `venv`를 사용하지 않는다면, following comando을 사용할 수 있습니다.

```bash
python -m hello_world.py 
```

또한, Python 3.x에서는 `print()` 함수를 사용하는 것만이 "Hello, World!"를 출력하도록합니다. 

Python 2.x에서는 `print`은 단순히 문장의wert을 의미한다.

```python
# Python 2.x
def main():
    # print() 함수는 그 안에 문자열을 입력해주면 해당 문자열이 화면에 표시됩니다.
    print "Hello, World!"  # Notice the quotes around the string

if __name__ == "__main__":
    main()
```


# 7. 명령형의 장점: 중간 처리 및 로깅

각 단계 사이에 **로깅, 데이터 변환, 검증** 등을 추가할 수 있습니다.

In [8]:
# 로깅이 포함된 체인
@chain
def logged_chatbot(values):
    print(f"[LOG] 입력값: {values}")

    # Step 1: 프롬프트 생성
    prompt = template.invoke(values)
    print(f"[LOG] 프롬프트 생성 완료")

    # Step 2: 모델 호출
    response = model.invoke(prompt)
    print(f"[LOG] 응답 길이: {len(response.content)}자")

    return response

# 테스트
print("=== 로깅 체인 테스트 ===")
response = logged_chatbot.invoke({'question': '안녕하세요!'})
print(f"\n최종 응답: {response.content}")

=== 로깅 체인 테스트 ===
[LOG] 입력값: {'question': '안녕하세요!'}
[LOG] 프롬프트 생성 완료
[LOG] 응답 길이: 39자

최종 응답: 안녕하세요! tôi很高兴能與你相遇! bạn cầnอะไร 도와드릴까요?


---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model='gpt-3.5-turbo')

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 선언형 vs 명령형 선택 가이드

| 상황 | 추천 방식 |
|------|----------|
| 단순한 순차 처리 | **LCEL** (`a \| b \| c`) |
| 조건부 분기 필요 | **@chain** (명령형) |
| 복잡한 에러 처리 | **@chain** (명령형) |
| 중간 로깅/검증 | **@chain** (명령형) |
| 빠른 프로토타이핑 | **LCEL** (간결함) |

## @chain 주의사항

1. **스트리밍**: 기본적으로 최종 결과만 반환됨. 중간 스트리밍이 필요하면 추가 처리 필요
2. **타입 힌트**: 입력/출력 타입을 명시하면 가독성 향상
3. **에러 처리**: try-except로 각 단계 에러를 세밀하게 제어 가능

```python
@chain
def safe_chatbot(values: dict) -> str:
    try:
        prompt = template.invoke(values)
        response = model.invoke(prompt)
        return response.content
    except Exception as e:
        return f"오류 발생: {e}"
```